In [7]:
import numpy as np
from tqdm import tqdm
import math
import itertools
import random

# Fibonacci Anyons

phi = (1 + np.sqrt(5)) / 2

sigma_1 = np.array([
    [np.exp(-4j * np.pi / 5), 0],
    [0, np.exp(3j * np.pi / 5)]
])
sigma_2 = np.array([
    [np.exp(4j * np.pi / 5) / phi, np.exp(-3j * np.pi / 5) / np.sqrt(phi)],
    [np.exp(-3j * np.pi / 5) / np.sqrt(phi), -1 / phi]
])
sigma_1_inv = np.linalg.inv(sigma_1)
sigma_2_inv = np.linalg.inv(sigma_2)

generators_map = {
    0: sigma_1,
    1: sigma_2,
    2: sigma_1_inv,
    3: sigma_2_inv
}
possible_indices = [0, 1, 2, 3]
inverse_mapping = {0: 2, 1: 3, 2: 0, 3: 1}

# Helper functions

def invert_seq(seq):
    """시퀀스의 역: 시퀀스의 각 요소를 inverse_mapping으로 대체하고, 역순으로 반환"""
    return [inverse_mapping[idx] for idx in reversed(seq)]

def apply_sequence(seq):
    """주어진 시퀀스 (리스트 of indices)에 해당하는 2x2 행렬의 곱을 반환"""
    M = np.eye(2, dtype=complex)
    for idx in seq:
        M = np.dot(M, generators_map[idx])
    return M

def distance(U, V):
    """
    Global Phase-Invariant Distance (SU(2) 기준)
    d(U, V) = sqrt( 1 - (|Tr(U V†)|/2)^2 )
    """
    return np.sqrt(1 - (np.abs(np.trace(np.dot(U, V.conj().T))) / 2)**2)

def basic_approximation(U, l0):
    """
    길이 l0 내 모든 가능한 시퀀스를 전수검색하여 target gate U에 가장 가까운 근사를 찾음.
    반환: (U_approx, seq, error)
    """
    best_seq = None
    best_U = None
    best_dist = float('inf')
    for seq in tqdm(itertools.product(possible_indices, repeat=l0), total=math.factorial(l0)):
        seq = list(seq)
        M = apply_sequence(seq)
        d = distance(U, M)
        if d < best_dist:
            best_dist = d
            best_seq = seq
            best_U = M
    return best_U, best_seq, best_dist

def group_commutator_decompose(Delta):
    """
    Delta ∈ SU(2)인 경우, Delta = V W V^(-1) W^(-1) 형태로 근사하기 위한
    V, W를 계산한다. 여기서는 Delta의 회전각 theta를 계산하고,
    임의의 두 직교축 (여기서는 x축과 y축)을 사용하여, theta/4 만큼 회전하도록 한다.
    """
    # SU(2) 행렬: Delta = cos(theta)I + i sin(theta) (n·σ)
    # theta = arccos(Tr(Delta)/2)
    theta = np.arccos(np.clip(np.trace(Delta).real / 2, -1, 1))
    if abs(theta) < 1e-6:
        return np.eye(2), np.eye(2)
    # 회전축: 임의로 [1, 0, 0]와 [0, 1, 0] 선택
    def exp_pauli(n, theta):
        I = np.eye(2, dtype=complex)
        n = np.array(n, dtype=float)
        n = n / np.linalg.norm(n)
        sigma_x = np.array([[0, 1], [1, 0]], dtype=complex)
        sigma_y = np.array([[0, -1j], [1j, 0]], dtype=complex)
        sigma_z = np.array([[1, 0], [0, -1]], dtype=complex)
        n_dot_sigma = n[0]*sigma_x + n[1]*sigma_y + n[2]*sigma_z
        return np.cos(theta)*I + 1j*np.sin(theta)*n_dot_sigma

    n = [1, 0, 0]
    n_prime = [0, 1, 0]
    V = exp_pauli(n, theta/4)
    W = exp_pauli(n_prime, theta/4)
    return V, W

def solovay_kitaev(U, n, l0):
    """
    U: target unitary (2x2 복소수 행렬)
    n: 재귀 깊이 (order of approximation)
    l0: 기본 근사 시퀀스의 길이 (0-order 근사)
    반환: (U_approx, seq, error)
      - U_approx: 근사된 2x2 단위행렬
      - seq: 기본 게이트 인덱스로 구성된 전체 시퀀스 (리스트 of int)
      - error: target U와 U_approx 사이의 Global Phase-Invariant error
    """
    print(f"현재 Unitary: {U}, 재귀 단계: {n}")
    if n == 0:
        U0, seq0, d0 = basic_approximation(U, l0)
        return U0, seq0, d0
    else:
        # 아래 order의 근사 U0를 구함
        U0, seq0, d0 = solovay_kitaev(U, n-1, l0)
        Delta = np.dot(U, U0.conj().T)
        V, W = group_commutator_decompose(Delta)
        # 재귀적으로 V와 W를 근사
        V_approx, seqV, dV = solovay_kitaev(V, n-1, l0)
        W_approx, seqW, dW = solovay_kitaev(W, n-1, l0)
        # V⁻^(-1)와 W^(-1)를 기본 시퀀스로 구함 (inverse 시퀀스)
        inv_seqV = invert_seq(seqV)
        inv_seqW = invert_seq(seqW)
        # U_new = V_approx W_approx V_approx^(-1) W_approx^(-1) U0
        # 전체 시퀀스는 seqV + seqW + inv_seqV + inv_seqW + seq0
        seq_new = seqV + seqW + inv_seqV + inv_seqW + seq0
        U_new = apply_sequence(seq_new)
        err = distance(U, U_new)
        return U_new, seq_new, err

def local_optimize_sequence(seq, U_target, iterations=100):
    """
    주어진 시퀀스 seq를 약간씩 변형(한 항목을 다른 가능한 인덱스로 대체)하여
    global phase-invariant error가 개선되는지 확인하는 간단한 hill-climbing 방식.
    U_target: target unitary
    반환: 개선된 시퀀스와 해당 에러값
    """
    best_seq = seq.copy()
    best_U = apply_sequence(best_seq)
    best_err = distance(U_target, best_U)
    
    for _ in range(iterations):
        # 시퀀스의 임의의 위치를 선택
        idx = random.randint(0, len(best_seq)-1)
        original_val = best_seq[idx]
        # 가능한 다른 게이트로 시도
        candidates = [x for x in possible_indices if x != original_val]
        improved = False
        for candidate in candidates:
            temp_seq = best_seq.copy()
            temp_seq[idx] = candidate
            temp_U = apply_sequence(temp_seq)
            temp_err = distance(U_target, temp_U)
            if temp_err < best_err:
                best_seq = temp_seq
                best_err = temp_err
                improved = True
                break  # 한 번 개선되면 break
        if not improved:
            # 해당 위치 변화에서 개선이 없으면 원래 값을 유지
            continue
    return best_seq, best_err

# Target unitary.
target_gate = np.array([[1, 0], [0, -1]], dtype=complex)

recursion_depth = 3
basic_length = 10

print("실행 전: 재귀 깊이 = {}, 기본 근사 길이 = {}".format(recursion_depth, basic_length))
U_approx, sequence, error = solovay_kitaev(target_gate, recursion_depth, basic_length)
print("\n[재귀적 Solovay-Kitaev 결과]")
print("Target Gate:\n", target_gate)
print("\nApproximated Gate:\n", U_approx)
print("\nApproximated Sequence (기본 게이트 시퀀스):\n", sequence)
print("\nFinal Global Phase-Invariant Error: {:.3e}".format(error))

# 국부 최적화 적용 (옵션)
opt_seq, opt_error = local_optimize_sequence(sequence, target_gate, iterations=200)
U_opt = apply_sequence(opt_seq)
print("\n[국부 최적화 적용 후]")
print("Optimized Sequence:\n", opt_seq)
print("Approximated Gate after Optimization:\n", U_opt)
print("Final Error after Optimization: {:.3e}".format(opt_error))

실행 전: 재귀 깊이 = 3, 기본 근사 길이 = 10


 29%|██▉       | 1048576/3628800 [00:15<00:37, 69729.38it/s]



[재귀적 Solovay–Kitaev 결과]
Target Gate:
 [[ 1.+0.j  0.+0.j]
 [ 0.+0.j -1.+0.j]]

Approximated Gate:
 [[ 0.54459341-0.76392207j  0.31037148-0.15333174j]
 [ 0.04991709-0.34256293j -0.55824445+0.75400402j]]

Approximated Sequence (기본 게이트 시퀀스):
 [3, 0, 0, 0, 3, 3, 0, 3, 0, 3, 0, 2, 1, 2, 2, 1, 2, 1, 1, 2, 1, 2, 1, 2, 1, 1, 2, 2, 2, 1, 0, 3, 3, 0, 3, 0, 0, 3, 0, 2, 1, 2, 2, 2, 1, 1, 2, 2, 2, 1, 1, 0, 1, 2, 2, 0, 0, 0, 1, 0, 1, 0, 1, 2, 2, 0, 0, 0, 1, 0, 2, 3, 2, 2, 2, 0, 0, 3, 2, 3, 2, 3, 2, 2, 2, 0, 0, 3, 2, 3, 1, 2, 2, 1, 0, 2, 2, 1, 1, 2, 3, 0, 0, 0, 3, 3, 0, 0, 0, 3, 0, 2, 1, 2, 2, 1, 2, 1, 1, 2, 3, 0, 0, 0, 3, 3, 0, 3, 0, 3, 0, 3, 3, 0, 3, 0, 0, 3, 0, 2, 1, 2, 1, 2, 1, 1, 2, 2, 2, 1, 0, 3, 3, 0, 0, 2, 3, 0, 0, 3, 1, 0, 1, 2, 2, 0, 0, 0, 1, 0, 1, 0, 1, 2, 2, 0, 0, 0, 1, 0, 2, 3, 2, 2, 2, 0, 0, 3, 2, 3, 2, 3, 2, 2, 2, 0, 0, 3, 2, 3, 1, 2, 2, 2, 1, 1, 2, 2, 2, 1, 1, 2, 2, 1, 1, 0, 0, 0, 0, 1, 3, 0, 0, 0, 3, 3, 0, 0, 0, 3, 3, 2, 2, 2, 2, 3, 3, 0, 0, 3, 1, 0, 0, 0, 0, 1, 2, 1, 1, 2, 2, 1, 1, 